In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from roboto.models import *

from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, LSTM, Reshape, Lambda, RepeatVector
from xgboost import XGBClassifier

In [3]:
INSTRUMENT_NAME = 'EU50_EUR'

In [4]:
instrument = Instrument.objects.get(name=INSTRUMENT_NAME)
candles = instrument.candles.values(
    'time',
    'open',
    'close',
)

In [22]:
frames = []

for instrument in Instrument.objects.all():
    columns = ('open', 'close')
    candles = list(instrument.candles.values_list('time', *columns))
    dataframe_columns = (['{}_{}'.format(instrument.name, c) for c in columns])
    tmp_df = pd.DataFrame(
        columns=['time', *dataframe_columns],
        data=candles,
    )
    if not len(frames):
        tmp_df['weekday'] = tmp_df.time.apply(lambda x: x.weekday)
        tmp_df['monthday'] = tmp_df.time.apply(lambda x: x.day)
        tmp_df['month'] = tmp_df.time.apply(lambda x: x.month)
        tmp_df['hour'] = tmp_df.time.apply(lambda x: x.hour)
    tmp_df.set_index(['time'], inplace=True)
    frames.append(tmp_df)
    
data = pd.concat(frames, axis=1, ignore_index=False)

first_full_field_index = None
for index, row in data.iterrows():
    if not row.isna().any():
        first_full_field_index = index
        break
first_full_field_index

data = data[first_full_field_index:]

data = data.fillna(method='ffill')

data.isna().any().any()

False

In [23]:
data.head()

,XAU_EUR_open,XAU_EUR_close,weekday,monthday,month,hour,UK100_GBP_open,UK100_GBP_close,XCU_USD_open,XCU_USD_close,...,US30_USD_open,US30_USD_close,GBP_HKD_open,GBP_HKD_close,USB05Y_USD_open,USB05Y_USD_close,NATGAS_USD_open,NATGAS_USD_close,USD_ZAR_open,USD_ZAR_close
time,,,,,,,,,,,,,,,,,,,,,
2018-08-15 07:00:00+00:00,1048.250,1048.299,2.0,15.0,8.0,7.0,7631.4,7616.3,2.63647,2.61730,...,25268.8,25251.2,9.98636,9.98213,113.677,113.663,2.956,2.954,14.26578,14.22086
2018-08-15 08:00:00+00:00,1048.393,1047.406,2.0,15.0,8.0,8.0,7616.7,7605.3,2.61742,2.60537,...,25251.7,25253.8,9.98209,9.97886,113.671,113.665,2.952,2.953,14.22086,14.34148
2018-08-15 09:00:00+00:00,1047.402,1047.240,2.0,15.0,8.0,9.0,7605.0,7578.3,2.60526,2.58997,...,25253.3,25212.2,9.97898,9.98586,113.661,113.692,2.952,2.954,14.34147,14.43672
2018-08-15 10:00:00+00:00,1047.276,1046.920,2.0,15.0,8.0,10.0,7578.8,7535.6,2.59008,2.59794,...,25212.2,25142.2,9.98566,9.97862,113.696,113.739,2.954,2.957,14.43654,14.59488
2018-08-15 11:00:00+00:00,1046.917,1048.398,2.0,15.0,8.0,11.0,7535.8,7529.3,2.59772,2.59447,...,25142.2,25131.0,9.97862,9.97206,113.749,113.764,2.958,2.942,14.59520,14.67718


In [24]:
def transform_categorial(df, categorial_columns):
    ret_df = df.copy()
    for feature in categorial_features:
        le = preprocessing.LabelBinarizer()    
        transformed_data = le.fit_transform(data[feature])
        new_columns = ['{}_{}'.format(feature, c) for c in le.classes_]
        for i in range(len(new_columns)):
            ret_df[new_columns[i]] = transformed_data[:,i]
    return ret_df.drop(categorial_columns, axis=1)

In [25]:
categorial_features = ['weekday', 'monthday', 'month', 'hour']
data = transform_categorial(data, categorial_features)

In [26]:
data.head()

,XAU_EUR_open,XAU_EUR_close,UK100_GBP_open,UK100_GBP_close,XCU_USD_open,XCU_USD_close,DE30_EUR_open,DE30_EUR_close,EUR_ZAR_open,EUR_ZAR_close,...,hour_14.0,hour_15.0,hour_16.0,hour_17.0,hour_18.0,hour_19.0,hour_20.0,hour_21.0,hour_22.0,hour_23.0
time,,,,,,,,,,,,,,,,,,,,,
2018-08-15 07:00:00+00:00,1048.250,1048.299,7631.4,7616.3,2.63647,2.61730,12392.8,12391.3,16.17726,16.11508,...,0,0,0,0,0,0,0,0,0,0
2018-08-15 08:00:00+00:00,1048.393,1047.406,7616.7,7605.3,2.61742,2.60537,12391.8,12383.6,16.11508,16.24266,...,0,0,0,0,0,0,0,0,0,0
2018-08-15 09:00:00+00:00,1047.402,1047.240,7605.0,7578.3,2.60526,2.58997,12383.8,12349.6,16.24264,16.34863,...,0,0,0,0,0,0,0,0,0,0
2018-08-15 10:00:00+00:00,1047.276,1046.920,7578.8,7535.6,2.59008,2.59794,12349.8,12270.6,16.34830,16.51943,...,0,0,0,0,0,0,0,0,0,0
2018-08-15 11:00:00+00:00,1046.917,1048.398,7535.8,7529.3,2.59772,2.59447,12270.2,12227.6,16.51980,16.61449,...,0,0,0,0,0,0,0,0,0,0


In [27]:
def create_df_with_diff(df, difference_values, columns, two_d=False):
    m, f = df.shape
    diff_df = df.copy()
    ret_df = None
    max_diff_value = 0
    for t in difference_values:
        new_columns = ['{}_{}'.format(c, t) for c in columns]
        tmp_df = df.drop(columns, axis=1)
        tmp_df[new_columns] = df[columns].diff(t)
        tmp_df = tmp_df.values
        if not two_d:
            tmp_df = tmp_df.reshape(m, 1, f)
        max_diff_value = max(max_diff_value, t)
        if ret_df is None:
            ret_df = tmp_df
        else:
            ret_df = np.hstack([ret_df, tmp_df])
    return ret_df[max_diff_value:]

In [28]:
import re
diff_columns = [c for c in data.columns if re.search('open|close', c)]

In [29]:
diff_values = [240, 216, 192, 168, 144, 120, 96, 72, 48, 24, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1,]
# diff_values = [10,9,8,7,6,5,4,3,2,1,]
max(diff_values)

240

In [30]:
X = create_df_with_diff(data, diff_values, diff_columns, two_d=False)[:-1]

Y = (data['{}_close'.format(INSTRUMENT_NAME)].diff(1)[1:] > 0) * 1
Y = Y[max(diff_values):]
Y.head()

time
2018-08-29 08:00:00+00:00    0
2018-08-29 09:00:00+00:00    0
2018-08-29 10:00:00+00:00    1
2018-08-29 11:00:00+00:00    1
2018-08-29 12:00:00+00:00    0
Name: EU50_EUR_close, dtype: int64

In [31]:
data.shape

(5261, 320)

In [32]:
data.head()

,XAU_EUR_open,XAU_EUR_close,UK100_GBP_open,UK100_GBP_close,XCU_USD_open,XCU_USD_close,DE30_EUR_open,DE30_EUR_close,EUR_ZAR_open,EUR_ZAR_close,...,hour_14.0,hour_15.0,hour_16.0,hour_17.0,hour_18.0,hour_19.0,hour_20.0,hour_21.0,hour_22.0,hour_23.0
time,,,,,,,,,,,,,,,,,,,,,
2018-08-15 07:00:00+00:00,1048.250,1048.299,7631.4,7616.3,2.63647,2.61730,12392.8,12391.3,16.17726,16.11508,...,0,0,0,0,0,0,0,0,0,0
2018-08-15 08:00:00+00:00,1048.393,1047.406,7616.7,7605.3,2.61742,2.60537,12391.8,12383.6,16.11508,16.24266,...,0,0,0,0,0,0,0,0,0,0
2018-08-15 09:00:00+00:00,1047.402,1047.240,7605.0,7578.3,2.60526,2.58997,12383.8,12349.6,16.24264,16.34863,...,0,0,0,0,0,0,0,0,0,0
2018-08-15 10:00:00+00:00,1047.276,1046.920,7578.8,7535.6,2.59008,2.59794,12349.8,12270.6,16.34830,16.51943,...,0,0,0,0,0,0,0,0,0,0
2018-08-15 11:00:00+00:00,1046.917,1048.398,7535.8,7529.3,2.59772,2.59447,12270.2,12227.6,16.51980,16.61449,...,0,0,0,0,0,0,0,0,0,0


In [33]:
data.tail()

,XAU_EUR_open,XAU_EUR_close,UK100_GBP_open,UK100_GBP_close,XCU_USD_open,XCU_USD_close,DE30_EUR_open,DE30_EUR_close,EUR_ZAR_open,EUR_ZAR_close,...,hour_14.0,hour_15.0,hour_16.0,hour_17.0,hour_18.0,hour_19.0,hour_20.0,hour_21.0,hour_22.0,hour_23.0
time,,,,,,,,,,,,,,,,,,,,,
2019-06-19 12:00:00+00:00,1198.102,1198.798,7421.8,7417.6,2.68291,2.68879,12342.8,12320.3,16.28107,16.27988,...,0,0,0,0,0,0,0,0,0,0
2019-06-19 13:00:00+00:00,1198.834,1199.378,7417.2,7425.3,2.68904,2.69179,12320.5,12316.0,16.27985,16.26922,...,0,0,0,0,0,0,0,0,0,0
2019-06-19 14:00:00+00:00,1199.415,1198.371,7425.0,7410.8,2.69191,2.68630,12316.2,12305.8,16.26950,16.28834,...,1,0,0,0,0,0,0,0,0,0
2019-06-19 15:00:00+00:00,1198.328,1199.466,7411.3,7405.8,2.68680,2.67782,12306.6,12307.1,16.28826,16.24381,...,0,1,0,0,0,0,0,0,0,0
2019-06-19 16:00:00+00:00,1199.504,1199.254,7405.8,7405.3,2.67807,2.67882,12306.8,12305.8,16.24388,16.25454,...,0,0,1,0,0,0,0,0,0,0


In [34]:
print (X.shape)
print (Y.shape)

(5020, 22, 320)
(5020,)


In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=False, test_size=.2)

In [36]:
X_test.shape

(1004, 22, 320)

In [37]:
def create_model():
    model = Sequential()
    model.add(LSTM(20, input_shape=X.shape[1:]))
    model.add(Dropout(0.1))
    model.add(Flatten(input_shape=X.shape[1:]))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))
    optimizer = optimizers.Adam(lr=0.0001,)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

model = create_model()

model.fit(
    X_train,
    Y_train,
    epochs=50,
    batch_size=256,
    validation_data=(X_test, Y_test),
    verbose=2,
    shuffle=False,
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 4016 samples, validate on 1004 samples
Epoch 1/50
 - 4s - loss: 0.6879 - acc: 0.5486 - val_loss: 0.6907 - val_acc: 0.5548
Epoch 2/50
 - 2s - loss: 0.6849 - acc: 0.5553 - val_loss: 0.6885 - val_acc: 0.5657
Epoch 3/50
 - 2s - loss: 0.6801 - acc: 0.5623 - val_loss: 0.6871 - val_acc: 0.5657
Epoch 4/50
 - 2s - loss: 0.6761 - acc: 0.5712 - val_loss: 0.6859 - val_acc: 0.5647
Epoch 5/50
 - 2s - loss: 0.6748 - acc: 0.5772 - val_loss: 0.6851 - val_acc: 0.5727
Epoch 6/50
 - 1s - loss: 0.6735 - acc: 0.5742 - val_loss: 0.6844 - val_acc: 0.5787
Epoch 7/50
 - 1s - loss: 0.6678 - acc: 0.5906 - val_loss: 0.6838 - val_acc: 0.5787
Epoch 8/50
 - 2s - loss: 0.6648 - acc: 0.5874 - val_loss: 0.6837 - val_acc: 0.5827
Epoch 9/50
 - 2s - loss: 0.6609 - acc: 0.6036 - val_loss: 0.6833 - val_acc: 0.5847
Epoch 10/50
 - 1s 

In [16]:
from learning.models import LearningModel

m = LearningModel.objects.create(
    name='test_test_2',
    description='test_test_2',
    type=LearningModel.TYPE_KERAS
)
m.save_model(model)

IntegrityError: (1062, "Duplicate entry 'test_test_2' for key 'name'")

In [1]:
m.model

NameError: name 'm' is not defined